In [7]:
import re


def camel_to_snake(name: str) -> str:
    return re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()


class ATest:
    @staticmethod
    def name() -> str:
        return 

In [8]:
str(ATest.name())

'ok'